<a href="https://colab.research.google.com/github/yoon777/DACON_GBT_competition/blob/main/NLP/BERTopic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bertopic
!pip install bertopic[visualization]
!pip install transformers
!pip install sentence_transformers
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.2 MB/s eta 0:00:00


In [2]:
from bertopic import BERTopic
import pandas as pd

In [33]:
df = pd.read_csv('extracted_text.csv')
df.head()

,page,text
0,1,U.S. Customs and Border Protection ◆ DEPARTMEN...
1,2,"2 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO...."
2,3,"3 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO...."
3,4,"4 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO...."
4,5,"5 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO...."


In [34]:
# 'text' 열에서 'U.S.'로 시작하는 행 필터링
df['remove_text'] = df['text'].str.replace("CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO. 38, SEPTEMBER 25, 2024", "", regex=False)
filtered_rows = df[df['remove_text'].str.contains(r'(^\d+\s+)?U\.S\.', regex=True)]
filtered = df[df['remove_text'].str.contains('◆', na=False)]

# 결과 출력
filtered_rows.remove_text

,remove_text
0,U.S. Customs and Border Protection ◆ DEPARTMEN...
1,2 SUPPLEMENTARY INFORMATION: Background The C...
2,3 restrictions under 19 U.S.C. 2602(a)(2). On...
15,"16 not limited to, plaques, stelae, memorial ..."
16,17 and promoting flexibility. CBP has determi...
17,"18 Authority: 5 U.S.C. 301; 19 U.S.C. 66, 120..."
18,19 DEPARTMENT OF THE TREASURY 19 CFR PART 12 ...
19,20 clusion of an agreement between the United...
39,40 a. Garments of the Cossack type are sewn f...
40,41 requires an agency to prepare and make ava...


In [35]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# NLTK 불용어 사전 다운로드
nltk.download('stopwords')
nltk.download('punkt')

# 불용어 리스트 로드
stop_words = set(stopwords.words('english'))

# 불용어 제거 함수 정의
def remove_stopwords(text):
    # 문장 단어 토큰화
    word_tokens = word_tokenize(text)
    # 불용어 제거
    filtered_words = [word for word in word_tokens if word.lower() not in stop_words]
    # 리스트를 다시 문장으로 합침
    return ' '.join(filtered_words)

# 데이터프레임의 특정 열에 불용어 제거 함수 적용
filtered_rows['filtered_text'] = filtered_rows['remove_text'].apply(remove_stopwords)

# 결과 확인
filtered_rows

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,page,text,remove_text,filtered_text
0,1,U.S. Customs and Border Protection ◆ DEPARTMEN...,U.S. Customs and Border Protection ◆ DEPARTMEN...,U.S. Customs Border Protection ◆ DEPARTMENT TR...
1,2,"2 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO....",2 SUPPLEMENTARY INFORMATION: Background The C...,2 SUPPLEMENTARY INFORMATION : Background Conve...
2,3,"3 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO....",3 restrictions under 19 U.S.C. 2602(a)(2). On...,3 restrictions 19 U.S.C . 2602 ( ) ( 2 ) . Nov...
15,16,"16 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO...","16 not limited to, plaques, stelae, memorial ...","16 limited , plaques , stelae , memorial stone..."
16,17,"17 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO...",17 and promoting flexibility. CBP has determi...,17 promoting flexibility . CBP determined docu...
17,18,"18 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO...","18 Authority: 5 U.S.C. 301; 19 U.S.C. 66, 120...","18 Authority : 5 U.S.C . 301 ; 19 U.S.C . 66 ,..."
18,19,"19 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO...",19 DEPARTMENT OF THE TREASURY 19 CFR PART 12 ...,19 DEPARTMENT TREASURY 19 CFR PART 12 CBP DEC....
19,20,"20 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO...",20 clusion of an agreement between the United...,20 clusion agreement United States another par...
39,40,"40 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO...",40 a. Garments of the Cossack type are sewn f...,40 . Garments Cossack type sewn long panels ( ...
40,41,"41 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO...",41 requires an agency to prepare and make ava...,41 requires agency prepare make available publ...


In [25]:
list_of_text = filtered_rows['filtered_text'].tolist()
list_of_text

['U.S. Customs Border Protection ◆ DEPARTMENT TREASURY 19 CFR PART 12 CBP DEC. 24–15 RIN 1515–AE83 IMPOSITION IMPORT RESTRICTIONS ARCHAEOLOGICAL ETHNOLOGICAL MATERIAL YEMEN AGENCY : U.S. Customs Border Protection , Department Homeland Security ; Department Treasury . ACTION : Final rule . SUMMARY : document amends U.S. Customs Border Protection ( CBP ) regulations reflect continuation import restrictions certain categories archaeological ethnological material Yemen , pursuant agreement United States Republic Yemen ( Agreement ) . restrictions originally imposed emergency basis CBP Decision 20–01 February 5 , 2020 , continue , various amendments , April 15 , 2029 . Designated List archaeo- logical ethnological material Yemen restrictions apply reproduced , certain modifications make consistent Agreement . DATES : Effective September 10 , 2024 . INFORMATION CONTACT : legal aspects , W. Richmond Beevers , Chief , Cargo Security , Carriers Restricted Merchandise Branch , Regulations Ruling

In [26]:
model = BERTopic()
topics, probabilities = model.fit_transform(list_of_text)

print('각 문서의 토픽 번호 리스트 :',len(topics))
print('첫번째 문서의 토픽 번호 :', topics[0])

model.get_topic_info()

각 문서의 토픽 번호 리스트 : 47
첫번째 문서의 토픽 번호 : 0


,Topic,Count,Name,Representation,Representative_Docs
0,-1,9,-1_see_new_zealand_fish,"[see, new, zealand, fish, plaintiffs, shepherd...","[79 July 1 , 2020 , Plaintiffs moved prelimina..."
1,0,22,0_cbp_19_ny_information,"[cbp, 19, ny, information, material, usc, cust...",[U.S. Customs Border Protection ◆ DEPARTMENT T...
2,1,16,1_katana_government_1592_court,"[katana, government, 1592, court, motion, judg...","[61 provided , remand , “ Katana able assert a..."


In [27]:
model.get_topic_info()['Count'].sum()

47

In [28]:
model.get_topic(-1)

[('see', 0.08041686458895045),
 ('new', 0.07905989312401716),
 ('zealand', 0.07093669048012459),
 ('fish', 0.061654577132397956),
 ('plaintiffs', 0.060523734643055334),
 ('shepherd', 0.054895840391565864),
 ('sea', 0.054895840391565864),
 ('united', 0.05134763494189299),
 ('states', 0.050170346776802266),
 ('comparability', 0.047854886402337204)]

## 페이지 별 묶어보기

In [39]:
# 5행씩 구간 설정
step = 5  # 5행씩 묶기
total_rows = len(filtered_rows)  # 전체 행의 수

# 구간을 자동으로 설정
ranges = [(start, min(start + step - 1, total_rows - 1)) for start in range(0, total_rows, step)]

# 새로운 열에 합친 텍스트 저장
merged_columns = []
for start, end in ranges:
    # 범위가 데이터프레임의 크기를 벗어나지 않는지 확인
    if start >= len(filtered_rows) or end >= len(filtered_rows):
        print(f"Warning: Range ({start}, {end}) is out of bounds for the given dataframe.")
        continue

    # 해당 범위의 텍스트를 합침
    merged_text = ' '.join(filtered_rows.iloc[start:end+1]['filtered_text'].to_list())
    merged_columns.append(merged_text)

# 새로운 DataFrame 생성 (묶인 텍스트를 기준으로)
merged_df = pd.DataFrame({'merged_text': merged_columns})
merged_df

,merged_text
0,U.S. Customs Border Protection ◆ DEPARTMENT TR...
1,"18 Authority : 5 U.S.C . 301 ; 19 U.S.C . 66 ,..."
2,42 ( b ) * * * State party Cultural property D...
3,"48 subject articles described heading 8509 , s..."
4,"53 agency , including whether information prac..."
5,"61 provided , remand , “ Katana able assert af..."
6,"66 Nonetheless , Federal Circuit made clear Ka..."
7,"71 Officer Hiyama , attempted communicate Kata..."
8,78 U.S.C . § 1371 ( ) . MMPA requires Secretar...
9,"Index Customs Bulletin Decisions Vol . 58 , . ..."


In [40]:
merged = merged_df['merged_text'].tolist()
merged

['U.S. Customs Border Protection ◆ DEPARTMENT TREASURY 19 CFR PART 12 CBP DEC. 24–15 RIN 1515–AE83 IMPOSITION IMPORT RESTRICTIONS ARCHAEOLOGICAL ETHNOLOGICAL MATERIAL YEMEN AGENCY : U.S. Customs Border Protection , Department Homeland Security ; Department Treasury . ACTION : Final rule . SUMMARY : document amends U.S. Customs Border Protection ( CBP ) regulations reflect continuation import restrictions certain categories archaeological ethnological material Yemen , pursuant agreement United States Republic Yemen ( Agreement ) . restrictions originally imposed emergency basis CBP Decision 20–01 February 5 , 2020 , continue , various amendments , April 15 , 2029 . Designated List archaeo- logical ethnological material Yemen restrictions apply reproduced , certain modifications make consistent Agreement . DATES : Effective September 10 , 2024 . INFORMATION CONTACT : legal aspects , W. Richmond Beevers , Chief , Cargo Security , Carriers Restricted Merchandise Branch , Regulations Ruling

In [41]:
model = BERTopic()
topics, probabilities = model.fit_transform(merged)

print('각 문서의 토픽 번호 리스트 :',len(topics))
print('첫번째 문서의 토픽 번호 :', topics[0])

model.get_topic_info()

각 문서의 토픽 번호 리스트 : 10
첫번째 문서의 토픽 번호 : -1


,Topic,Count,Name,Representation,Representative_Docs
0,-1,10,-1_katana_19_usc_united,"[katana, 19, usc, united, states, court, gover...","[66 Nonetheless , Federal Circuit made clear K..."


In [42]:
model.get_topic_info()['Count'].sum()

10

In [43]:
model.get_topic(0)

False